In [2]:
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "items"

In [3]:
from pymongo import MongoClient
from collections import defaultdict


client = MongoClient(MONGO_URI)
db = client[DB_NAME]
user_collection = db["user"]


# 유저별 상호작용한 아이템 ID 수집
user_item_counts = defaultdict(int)
duplicate_items = defaultdict(list)

# 모든 유저 순회
for user in user_collection.find({}, {"_id": 1, "items.itemnum": 1}):
    user_id = user["_id"]
    itemnums = [item["itemnum"] for item in user.get("items", [])]

    # 아이템 등장 횟수 카운트
    item_counts = defaultdict(int)
    for itemnum in itemnums:
        item_counts[itemnum] += 1

    # 중복된 아이템이 있는 경우 저장
    for itemnum, count in item_counts.items():
        if count > 1:
            duplicate_items[user_id].append((itemnum, count))

# 결과 출력
import pandas as pd
df_duplicates = pd.DataFrame(
    [(user_id, itemnum, count) for user_id, items in duplicate_items.items() for itemnum, count in items],
    columns=["UserID", "ItemNum", "DuplicateCount"]
)

In [10]:
user_data = user_collection.find_one({"_id": "4"})
user_data

{'_id': '4',
 'reviewerID': 'A3JVF9Y53BEOGC',
 'password': '1234',
 'items': [{'itemnum': 6635,
   'asin': '6305252599',
   'reviewText': 'The video and DVD versions of this film are NOT the same as was seen in the theatres. Several minutes of footage were added that significantly altered the film, especially it\'s ending. It is NOT a Director\'s cut. It was probably done to assuage a cast member of the "Deep Space 9" TV series, whose contributions to this film had ended up on the cutting room floor. The original (better) version should have been available on this disc.',
   'overall': 3.0,
   'summary': 'Bring back the theatrical version!',
   'unixReviewTime': 983577600},
  {'itemnum': 10418,
   'asin': 'B00005MEPO',
   'reviewText': "Ken Burns doing Frank Lloyd Wright. Expectations among fans of Wright's work were very high. But Burns fell woefully short in capturing the essence of the man and in explaining what makes his architecture great (and what makes the ideas behind it still 

In [12]:
user_data.items

<function dict.items>

In [4]:
df_duplicates

,UserID,ItemNum,DuplicateCount
0,4,26573,2
1,6,67433,2
2,7,5903,2
3,7,25788,2
4,7,25873,2
...,...,...,...
124340,311088,45637,2
124341,311100,83642,2
124342,311100,83643,2
124343,311110,82936,2
